# 1. Descarga y adaptación de la información

## Imports & Settings

In [384]:
import warnings
warnings.filterwarnings('ignore')

In [385]:
%matplotlib inline

from datetime import datetime
import pandas as pd
import pandas_datareader.data as web
from sklearn.preprocessing import scale
# replaces pyfinance.ols.PandasRollingOLS (no longer maintained)
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

In [386]:
import numpy as np

In [387]:
sns.set_style('whitegrid')
idx = pd.IndexSlice

In [388]:
import yfinance as yf

In [389]:
normaliza=True #normalizamos por volatilidad
neutraliza=False #normalizado cross sectional media y vola

In [390]:
DATA_STORE = '../data/assets.h5'

In [391]:
START = 1990
END = 2024

# Descarga de precios

In [ ]:
ticker_list = ['XLE', 'XLB', 'XLI', 'XLK', 'XLF', 
               'XLP', 'XLY', 'XLV', 'XLU', 'IYR', 'VOX', 'SPY', 'QQQ']
 
# Here we use yf.download function
data = yf.download(
     
    # passes the ticker
    tickers=ticker_list,
     
    # used for access data[ticker]
    group_by='ticker',
 
)

In [ ]:
#apilamos tickers
data = (data
                .stack(-2))
data

In [395]:
data = data.rename_axis(['date', 'ticker'])

In [396]:
# Reordenar y renombrar las columnas directamente
new_order = ['Open', 'High', 'Low','Close','Volume', 'Adj Close']
new_names = ['open', 'high', 'low','close','volume', 'Adj Close']

# Reordenar las columnas
prices = data[new_order]

# Renombrar las columnas
prices.columns = new_names


In [397]:
prices=prices.sort_index()


In [398]:
# Crear un nuevo DataFrame sin entradas duplicadas en el índice
prices = prices.loc[~prices.index.duplicated(keep='first')]


In [ ]:
prices.info()

In [ ]:
print(prices.index.unique('ticker'))

In [ ]:
num_ticker = prices.index.get_level_values('ticker').nunique()

print(f'El índice "ticker" tiene {num_ticker} elementos únicos.')


In [402]:
prices = prices.swaplevel(0, 1)

In [ ]:
primera_fecha_no_nula = (
    prices.notnull()                                      # Identifica valores no nulos
    .any(axis=1)                                     # Filtra filas con al menos un valor no nulo
    .groupby(level='ticker')                         # Agrupa por 'Ticker'
    .apply(lambda x: x[x].index.get_level_values('date').min())  # Obtiene la primera fecha
)
primera_fecha_no_nula

# Adaptamos VOX y IYR

In [ ]:
np.mean(prices.xs('IYR', level='ticker').loc['2000-06-19':'2001-06-20']['close'] / prices.xs('XLF', level='ticker').loc['2000-06-19':'2001-06-20']['close'])

In [ ]:
prices = prices.sort_index(level=1)

spy_rows = prices.xs('XLF', level='ticker').head(375)
factor_XLF = 1.6697486934700732
spy_rows['close'] = spy_rows['close'] * factor_XLF
spy_rows['ticker'] = 'IYR'
spy_rows = spy_rows.set_index('ticker', append=True)
spy_rows = spy_rows.swaplevel(0, 1)
prices = pd.concat([prices, spy_rows])
prices

In [ ]:
np.mean(prices.xs('VOX', level='ticker').loc['2004-09-29':'2005-09-29']['close'] / prices.xs('QQQ', level='ticker').loc['2004-09-29':'2005-09-29']['close'])

In [ ]:
spy_rows = prices.xs('QQQ', level='ticker').head(1448-51)
spy_rows_1 = spy_rows.head(900)
spy_rows_2 = spy_rows.tail(1448-51-900)
spy_rows_1

In [ ]:
prices = prices.sort_index(level=1)  # Nivel 0 normalmente corresponde a las fechas

spy_rows = prices.xs('QQQ', level='ticker').head(1448-51)
spy_rows_1 = spy_rows.head(900)
spy_rows_2 = spy_rows.tail(1448-51-900)


factor_VOX = 1.4035602852516436
spy_rows_2['close'] = (spy_rows_2['close']) * factor_VOX
spy_rows_1['close'] = (spy_rows_1['close']) * 0.75
spy_rows = pd.concat([spy_rows_1, spy_rows_2])
spy_rows['ticker'] = 'VOX'
spy_rows = spy_rows.set_index('ticker', append=True)
spy_rows = spy_rows.swaplevel(0, 1)
prices = pd.concat([prices, spy_rows])
prices

In [409]:
#eliminamos spy
prices = prices.drop(index='SPY', level=0)
#eliminamos spy
prices = prices.drop(index='QQQ', level=0)


In [410]:
prices = prices.sort_index(level=list(range(len(prices.index.names))))  # Sort all levels

In [ ]:
#guardamos los datos de ohlcv
prices.index = prices.index.set_levels([prices.index.levels[0], prices.index.levels[1].tz_localize(None)])


with pd.HDFStore(DATA_STORE) as store:
    store.put('data_close', prices.sort_index())
    print(store.info())

In [ ]:
#hacemos unstack de close sólo
prices= (prices.loc[idx[:,str(START):str(END)], 'close']
              .unstack('ticker'))
prices

In [ ]:
prices.loc['2000':].plot(figsize=(20,10))

In [414]:
tiene_indices_duplicados = prices.index.duplicated().any()


In [ ]:
tiene_indices_duplicados

In [416]:
monthly_prices_real = prices.resample('M').last()#para que las betas de fama french esten alineadas
monthly_prices = prices.resample('W').last()

In [ ]:
outlier_cutoff = 0.01
data = pd.DataFrame()
lags =  [1, 2, 3, 6, 12, 52] #para semanas
for lag in lags:
    data[f'return_{lag}m'] = (monthly_prices
                           .pct_change(lag)
                           .stack()
                           .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1-outlier_cutoff)))
                           .add(1)
                           .pow(1/lag)
                           .sub(1)
                           )
data = data.swaplevel().dropna()
data.info()

In [419]:
#para fama frech
return_1m_real=monthly_prices_real.pct_change()

In [ ]:
return_1m_real=return_1m_real.stack().swaplevel().dropna()
return_1m_real.head()

In [421]:
return_1m_real.name='return_1m'

In [ ]:
min_obs = 52*10 #semanal
nobs = data.groupby(level='ticker').size()
keep = nobs[nobs>min_obs].index

data = data.loc[idx[keep,:], :]
data.info()

In [ ]:
data.describe()

In [425]:
data2=data.copy()

## Nomalizado de retornos

In [426]:
def normalize_by_rolling_std(series):
    return series / series.rolling(52).std().shift(1)

In [427]:
# normaliza=True

In [428]:
if normaliza==True:
    lags = [1, 2, 3, 6, 12, 52] #para semanas
    for lag in lags:
           data[f'return_{lag}m'] = (data.groupby(level='ticker')[f'return_{lag}m']
                             .transform(normalize_by_rolling_std)
                             )

In [429]:
# Función para neutralizar (normalizar) los retornos por cada fecha
def neutralize(group):
    return (group - group.mean()) / group.std()


In [430]:
neutraliza = False

In [431]:
if neutraliza==True:
    lags = [1, 2, 3, 6, 12, 52] #para semanas
    for lag in lags:
           data[f'return_{lag}m'] = (data.groupby(level='date')[f'return_{lag}m']
                             .transform(neutralize))

In [ ]:
data.tail(20)

In [ ]:
data2.tail(20)

In [ ]:
# cmap = sns.diverging_palette(10, 220, as_cmap=True)
sns.clustermap(data.corr('spearman'), annot=True, center=0, cmap='Blues');

## Rolling Factor Betas

In [ ]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start='1999')[0].drop('RF', axis=1)
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample('M').last().ffill().div(100) #semanal

factor_data.index.name = 'date'
factor_data.info()

### importante metemos el retorno sin adaptar para la regresión

In [ ]:
data2['return_1m'].name

In [ ]:
factor_data = factor_data.join(return_1m_real).sort_index()
factor_data.info()

In [439]:
T = 24
betas = (factor_data.groupby(level='ticker',
                             group_keys=False)
         .apply(lambda x: RollingOLS(endog=x.return_1m,
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(T, x.shape[0]-1))
                .fit(params_only=True)
                .params
                #.drop('const', axis=1)
               ))

In [ ]:
betas.describe().join(betas.sum(1).describe().to_frame('total'))

In [ ]:
cmap = sns.diverging_palette(10, 220, as_cmap=True)
sns.clustermap(betas.corr(), annot=True, cmap=cmap, center=0);

In [ ]:
betas.info()


In [ ]:
betas.loc['XLK',"2002"].head(10)

In [ ]:
data = (data
        .join(betas
              .groupby(level='ticker')
              .shift(1)))#hacemos shift pq lo conoceremos un mes despues
data.info()

In [446]:
factors=['const','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

In [ ]:
data.loc[:, factors] = data.groupby('ticker')[factors].transform(lambda x: x.ffill())
data.info()

In [448]:
#incorporamos los cambios en los datos de betas
for columna in factors:
 data[columna + '_diff'] = data[columna].diff().replace(0, np.nan).ffill()

## Momentum factors

In [449]:
for lag in [2,3,6,12, 52]: #para semanas
    data[f'momentum_{lag}'] = data[f'return_{lag}m'].sub(data.return_1m)
data[f'momentum_3_12'] = data[f'return_12m'].sub(data.return_3m)

## Date Indicators

In [450]:
dates = data.index.get_level_values('date')
data['month'] = dates.month

## Sector

In [454]:
# Crear una Serie con el índice del DataFrame y los valores del nivel 'ticker'
ticker_series = pd.Series(data.index.get_level_values('ticker'), index=data.index)

# Usar esta Serie para llenar los valores NA
data['sector'] = ticker_series

In [ ]:
primera_fecha_no_nula = (
    data.notnull()                                      # Identifica valores no nulos
    .any(axis=1)                                     # Filtra filas con al menos un valor no nulo
    .groupby(level='ticker')                         # Agrupa por 'Ticker'
    .apply(lambda x: x[x].index.get_level_values('date').min())  # Obtiene la primera fecha
)
primera_fecha_no_nula

In [ ]:
data[data['sector'].isna()].index.get_level_values(0).unique()


## Lagged returns

In [ ]:
for t in range(1, 7):
    data[f'return_1m_t-{t}'] = data.groupby(level='ticker').return_1m.shift(t)
data.info()

## Target: Holding Period Returns

In [460]:
for t in [1,2,3,6,12]:
    data[f'target_{t}m'] = data.groupby(level='ticker')[f'return_{t}m'].shift(-t)

In [ ]:
cols = ['target_1m',
        'target_2m',
        'target_3m', 
        'return_1m',
        'return_2m',
        'return_3m',
        'return_1m_t-1',
        'return_1m_t-2',
        'return_1m_t-3']

data[cols].dropna().sort_index().head(10)

# Variables Macroeconómicas

In [463]:
indicators = (['JHDUSRGDPBR', 'T10Y3M', 'BAMLC0A0CM','BAMLH0A0HYM2','BAMLHE00EHYIOAS', 'UMCSENT','UNRATE','GDPC1','DCOILWTICO',
               'CORESTICKM159SFRBATL','USSLIND',
               'VIXCLS','OVXCLS','ICSA','MARTSMPCSM44000USS','RSXFS','TREAST','DGS1', 'USREC', 'T10Y2Y'])
var_names = (['recession', 'yield_curve', 'corp_oas', 'hy_oas','eu_hy_oas', 'sentiment','empleo','real_gdp','oil',
              'inflacion','leading',
              'vix','vixoil','weekjobclaims','retail_sales_percent','retail_sales','us_asset_balance','1y_yield', 'USREC', 'curva_tipos'])

In [464]:
features = var_names[1:]
label = var_names[0]

In [465]:
var_display = (['Recession', 'Yield Curve', 'corp_oas', 'hy_oas','eu_hy_oas' , 'Sentiment','empleo','real_gdp','oil','inflacion',
              'leading','vix','vixoil','weekjobclaims','retail_sales_percent',
                'retail_sales','us_asset_balance','1y_yield', 'USREC', 'curva_tipos'])
col_dict = dict(zip(var_names, var_display))

In [466]:
data_fred = (web.DataReader(indicators, 'fred', 1980, END+1)
        
        
        .resample('W') 
        .last()
        .ffill())  
data_fred.columns = var_names

In [ ]:
data_fred.info()

In [468]:
data_fred.index.name = 'date'

In [470]:
#incorporamos los cambios en los datos
for columna in data_fred.columns:
    data_fred[columna + '_diff'] = data_fred[columna].diff().replace(0, np.nan).ffill()
    data_fred[columna + '_chg'] = data_fred[columna].pct_change().replace(0, np.nan).ffill()

In [471]:
#eliminamos algunas variables que tienen mucha dependencia del nivel historico
data_fred = data_fred.drop(['empleo','us_asset_balance'], axis=1)

In [472]:
#para hacer bbfill sólo hasta que encuentre un primer valor
for columna in data_fred.columns:
    # Verificar si la columna tiene NaN al inicio
    if data_fred[columna].isna().iloc[0]:  
        # Obtiene el primer valor no NaN de la columna
        primer_valor = data_fred[columna].dropna().iloc[0]
        # Rellena los NaN iniciales con el primer valor no NaN
        data_fred[columna][:data_fred[columna].first_valid_index()] = primer_valor


In [474]:
#data_fred.index = data_fred.index.to_timestamp()
data_fred.index.name = 'date'

In [ ]:
data = (data
        .join(data_fred))
data.info()

In [476]:
def determinar_fase(row):
    if row['USREC'] == 1:
        return 'Recesión'
    elif row['USREC'] == 0 and row['curva_tipos'] < 0:
        return 'Desaceleración'
    elif row['USREC'] == 0 and row['real_gdp'] > row['pib_media_5']:
        return 'Expansión'
    elif row['USREC'] == 0 and row['real_gdp'] <= row['pib_media_5']:
        return 'Recuperación'
    return 'Desconocida'

data['pib_media_5'] = data['real_gdp'].rolling(window=12).mean()

# Crear columna 'fase'
data['fase'] = data.apply(determinar_fase, axis=1)


# Guardamos la información

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('engineered_features', data.sort_index())
    store.put('data_raw', data2.sort_index())#antes de normalizado de retornos
    print(store.info())